In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Clases que queremos clasificar
CLASS_NAMES = ['hamburger', 'pizza', 'ice_cream', 'tiramisu']
NUM_CLASSES = len(CLASS_NAMES)

In [ ]:
# Cargar el dataset con filtrado de las clases de comida chatarra
def load_filtered_dataset():
    _, info = tfds.load('food101', with_info=True, as_supervised=True)
    label_names = info.features['label'].names

    class_indices = [label_names.index(cls) for cls in CLASS_NAMES]

    def filter_classes(image, label):
        return tf.reduce_any(tf.equal(label, class_indices))

    (train_raw, test_raw), info = tfds.load(
        'food101',
        split=['train', 'validation'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True
    )

    train_data = train_raw.filter(filter_classes)
    test_data = test_raw.filter(filter_classes)

    return train_data, test_data, info

# Cargar datos filtrados
datos_entrenamiento_raw, datos_pruebas_raw, metadatos = load_filtered_dataset()

In [ ]:
# Preprocesamiento
IMG_SIZE = 128

def preprocesar(imagen, etiqueta):

    label_names = metadatos.features['label'].names
    class_indices = [label_names.index(cls) for cls in CLASS_NAMES]
    etiqueta = tf.argmax(tf.cast(tf.equal(etiqueta, class_indices), tf.int32))

    # Procesamiento de imagen
    imagen = tf.image.resize(imagen, (IMG_SIZE, IMG_SIZE))
    imagen = tf.cast(imagen, tf.float32) / 255.0
    return imagen, etiqueta

# Aplicar preprocesamiento
datos_entrenamiento = datos_entrenamiento_raw.map(preprocesar)
datos_pruebas = datos_pruebas_raw.map(preprocesar)

# Optimización 1: Aumento de datos
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

datos_entrenamiento = datos_entrenamiento.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Preparar batches
TAMANO_LOTE = 32
datos_entrenamiento = datos_entrenamiento.shuffle(3000).batch(TAMANO_LOTE).prefetch(tf.data.AUTOTUNE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Modelo con dropout y aumento de datos
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compilar
modelo.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen
modelo.summary()

# Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entrenamiento
print("\nEntrenando modelo...")
historial = modelo.fit(
    datos_entrenamiento,
    epochs=30,
    validation_data=datos_pruebas,
    callbacks=[early_stopping]
)


Entrenando modelo...
Epoch 1/15
     94/Unknown 214s 2s/step - accuracy: 0.2950 - loss: 1.4602

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


94/94 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.2955 - loss: 1.4592 - val_accuracy: 0.4730 - val_loss: 1.2242
Epoch 2/15
94/94 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.4284 - loss: 1.2534 - val_accuracy: 0.5380 - val_loss: 1.1642
Epoch 3/15


KeyboardInterrupt: 

In [ ]:
modelo.save('chatarra.h5')

In [ ]:
!pip install tensorflowjs -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.31.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
!mkdir -p carpeta_salida
!tensorflowjs_converter --input_format keras chatarra.h5 carpeta_salida

2025-05-05 06:48:57.799182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746427738.164458   24050 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746427738.267634   24050 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file
